<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="gemma3:12b" # "llama3.2"
model_2_id="deepseek-r1:8b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [2]:
!pip install openai --quiet

In [3]:
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {model_2_id}")

In [4]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [5]:
def call(system_prompt: str, message: str, model: str = modelid ) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [6]:
def marketing_campaign_agent(product : str):
  """

     Your task is to create an compelling marketing campaign plan for the product.

    ## Product Description
    {product}

     ## Marketing Campaign Plan:L
     Your marketing campaign plan:

  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [7]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

  Attached is a product descipription of a new product we are launching.

  Also attached is a marketing campaign plan for the product launch.

  Your task is to read the marketing campaign plan and assess wether this is good enough to get finalized or wether it would benefit from another revision.

  Answer with TRUE or FALSE. add nothing else to your response.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Your Assessment
  Your Assessment:

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(product=product, text=marketing_text) , model=model_2_id)
  print (result)
  return result.strip().lower() == "true"

In [8]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """

  Attached is a product descipription of a new product we are launching.

  Also attached is a draft marketing campaign plan for the product launch.

  Provide a critique of this draft marketing plan in preparation of the final version.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Critique
  Your critique:

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [9]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """

  Attached are 3 documents
    - a product descipription of a new product we are launching.
    - a draft marketing campaign plan for the product launch.
    - a critique of this draft marketing plan in preparation of the final version

  Please write another version of the marketing campaign plan taking into account the feedback in the critique.

  ## Product Description
  {product}

  ## Marketing Text
  {text}

  ## Critique
  {refinements}

  ## Final Marketing Text
  Your final version of the marketing campaign plan:


  """
  return call ("your are a marketing campaign export.", refine_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [10]:
product = "iPhone 15"

iteration=1
marketing_text = marketing_campaign_agent(product)
while not ready_validation_agent(marketing_text) and iteration<5:
  print (f"ITERATION: {iteration}" )
  refinements = refinements_agent(product, marketing_text)
  next_text_iteration = refine_agent(product, marketing_text, refinements)
  print (refinements)
  iteration+=1
  marketing_text = next_text_iteration


FALSE
ITERATION: 1
Okay, this is a *really* solid draft marketing campaign plan! It's well-structured, covers all the bases, and has a good understanding of both the product and the target audience.  Here's a detailed critique, broken down into strengths, areas for improvement, and specific suggestions. I’ll also include a ‘Big Picture’ section at the end to address potential overarching risks/opportunities.

**I. Strengths – What Works Well**

*   **Clear Narrative & Theme ("Capture Your Story"):** The tagline is excellent. It's evocative, memorable, and directly ties to a core benefit (connecting through shared experiences).  The entire campaign plan is built around this central narrative, which is crucial for cohesion.
*   **Well-Defined Target Audience Segments:**  The segments are distinct and have clear motivations for wanting an iPhone 15.  Understanding these motivations is key to crafting effective messaging.
*   **SMART Objectives:**  The objectives are generally well-defined

In [ ]:
print (next_text_iteration)


In [ ]:
print (marketing_text)
print ("###### critique")
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)

In [11]:
print (marketing_text)

Okay, here's a revised marketing campaign plan incorporating your thorough critique. I've addressed each point, aiming for not just correction but strategic improvement. This version aims to be even more robust, detailed, and forward-thinking. **I'm also including a final checklist at the very end for easy reference before launch.**

**Marketing Campaign Plan: iPhone 15 – “Capture Your Story, Own Your Moment” (Version 2.0 – Incorporating Critique)**

**1. Executive Summary:** (No change - Solid as is)

**2. Target Audience (Refined Segments):** (No change - Excellent segmentation)

**3. Campaign Objectives (SMART Goals - Enhanced & Methodologically Sound):**

*   **Increase Brand Awareness:** Achieve a 20% increase in social media mentions & demonstrable positive sentiment (qualitative analysis of comments, reviews) surrounding the iPhone 15 within Q1. Track with Brandwatch/Similar tools. *Methodology: Track mentions and sentiment using Brandwatch. Qualitative analysis to be performed 